# GTFS Loader Example

In [ ]:
from pathlib import Path
from srai.loaders import GTFSLoader
import gtfs_kit as gk
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from srai.utils.constants import WGS84_CRS

## Download an example GTFS feed from Wroclaw, Poland

In this notebook we use the GTFS feed for Wroclaw, Poland as an example, which is available in Wroclaw's open data repository[1]. This download uses transitfeeds.com[2] to download the feed, but you can also download the feed directly from the Wroclaw open data repository.

1. https://www.wroclaw.pl/open-data/dataset/rozkladjazdytransportupublicznegoplik_data
2. https://transitfeeds.com/p/mpk-wroc-aw/663

In [ ]:
! wget -O files/example_2.zip https://transitfeeds.com/p/mpk-wroc-aw/663/20221221/download

In [ ]:
wroclaw_gtfs = Path().resolve() / "files" / "example_2.zip"

## Peek at the feed using `gtfs_kit` directly

In [ ]:
feed = gk.read_feed(wroclaw_gtfs, dist_units="km")

stops_df = feed.stops[["stop_id", "stop_lat", "stop_lon"]].set_index("stop_id")
stops_df["geometry"] = stops_df.apply(lambda row: Point(row["stop_lon"], row["stop_lat"]), axis=1)

stops_gdf = gpd.GeoDataFrame(
    stops_df,
    geometry="geometry",
    crs=WGS84_CRS,
)

stops_gdf.plot(markersize=1)

## Use GTFSLoader to load stops statistics from the feed

In [ ]:
gtfs_loader = GTFSLoader()
trips_gdf = gtfs_loader.load(wroclaw_gtfs)

print(trips_gdf.columns)